# Ejemplo del uso de Google Colaboratory

Ejemplo de cómo usar Google [Colab](https://colab.research.google.com), el servicio gratuito en la nube de Google para desarrolladores de inteligencia artificial. Con Colab, puede desarrollar aplicaciones de aprendizaje profundo usando Tarjetas Graficas GPU de forma gratuita.

Con Colab se puede usar el Google Drive para almacenar los Notebook de Jupyter y los datos necesarios

### Instalar librerías para acceder al Google Drive

In [18]:
# Instalar la libreria FUSE.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

gpg: keybox '/tmp/tmpft4c3y_p/pubring.gpg' created
gpg: /tmp/tmpft4c3y_p/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


### Generar los tokens de autorización para Colab

In [ ]:
# Generar los tokens de autorización para Colab
from google.colab import auth
auth.authenticate_user()

### Generar credenciales para la librería FUSE

In [20]:
# Generar credenciales para la librería FUSE
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

··········


### Crear un directorio y montar Google Drive usando ese directorio

In [21]:
# Crear un directorio y montar Google Drive usando ese directorio
!mkdir -p drive
!google-drive-ocamlfuse drive

print ('Archivos en Drive:')
!ls drive/ColabNotebooks    # ColabNotebooks es el nombre del directorio en Google Drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option
Archivos en Drive:
01-Introduccion.ipynb				Fashion-MNIST.ipynb
21-RedesNeuronales-3.ipynb			figuras
22-RedesNeuronales-4.ipynb			Generador_Cervantes_2.ipynb
23-RedNeuronalConvolucional-1.ipynb		keras-mnist-tutorial
24-RedNeuronalConvolucional-2.ipynb		mnist_cnn.py
25-RedNeuronalConvolucional-3.ipynb		pesos-v1.01-14.4789.hdf5
26-RedNeuronalConvolucional-4 (32d4ba2a).ipynb	pesos-v1.03-14.3032.hdf5
26-RedNeuronalConvolucional-4.ipynb		pesos-v1.20-14.1243.hdf5
30.EjemploUsoColaboratory.ipynb			pesos-v1.81-13.7262.hdf5
Breast_Cancer.ipynb				Pytorch_Tutorial.ipynb
deep_learning_turkey.ipynb			ReconocimientoEmociones
DonQuijote.txt					ReconocimientoEmosiones.ipynb
DonQuixote.txt					Titanic.csv
entrenamiento-v1.log


## Ejemplo de una Red Neuronal Convolucional con la data CIFAR-10

In [ ]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau
from keras.utils import to_categorical

#### Cargar los datos

In [23]:
# Conjunto de datos de entrenamiento y prueba:
(x_entrenamiento, y_entrenamiento), (x_prueba, y_prueba) = cifar10.load_data()
print('Dimensiones de x_entrenamiento:', x_entrenamiento.shape)
print(x_entrenamiento.shape[0], 'ejemplos de entrenamiento')
print(x_prueba.shape[0], 'ejemplos de prueba')

Dimensiones de x_entrenamiento: (50000, 32, 32, 3)
50000 ejemplos de entrenamiento
10000 ejemplos de prueba


#### Convertir las etiquetas

In [ ]:
# Convertir vectores de clases a matrices binarias.
num_clases = 10

y_entrenamiento = keras.utils.to_categorical(y_entrenamiento, num_clases)
y_prueba = keras.utils.to_categorical(y_prueba, num_clases)

#### Normalizar las entradas a valores entre 0 y 1

In [ ]:
x_entrenamiento = x_entrenamiento.astype('float32')
x_prueba = x_prueba.astype('float32')
x_entrenamiento /= 255
x_prueba /= 255

#### Definir el esquema de aumento de datos

In [ ]:
# pre-procesamiento de las imágenes para aumentarlas
datagen = ImageDataGenerator(
    rotation_range=15,  # rotar aleatoriamente las imágenes entre 0 y 180 grados 
    width_shift_range=0.1,  # mover aleatoriamente (horizontalmente) las imágenes (fracción del ancho total)
    height_shift_range=0.1,  # mover  aleatoriamente (verticalmente) las imágenes (fracción del alto total)
    horizontal_flip=True,  # volterar aleatoriamente las imágenes (horizontalmente)
    vertical_flip=False)  # volterar aleatoriamente las imágenes (verticalmente)

### Definir la arquitectura de la Red Neuronal Convolucional

In [27]:
# Fijar eje para la BatchNormalization en 3
bn_axis = 3

modelo = Sequential()

modelo.add(Conv2D(32, (3, 3), padding='same', input_shape=x_entrenamiento.shape[1:], name='conv1'))
modelo.add(BatchNormalization(axis=3, name='bn_conv1'))
modelo.add(Activation('relu'))
modelo.add(Conv2D(32, (3, 3), name='conv2'))
modelo.add(BatchNormalization(axis=3, name='bn_conv2'))
modelo.add(Activation('relu'))
modelo.add(MaxPooling2D(pool_size=(2, 2)))

modelo.add(Conv2D(64, (3, 3), padding='same', name='conv3'))
modelo.add(BatchNormalization(axis=3, name='bn_conv3'))
modelo.add(Activation('relu'))
modelo.add(Conv2D(64, (3, 3), name='conv4'))
modelo.add(BatchNormalization(axis=3, name='bn_conv4'))
modelo.add(Activation('relu'))
modelo.add(MaxPooling2D(pool_size=(2, 2)))

modelo.add(Flatten())
modelo.add(Dense(512, name='fc1'))
modelo.add(BatchNormalization(axis=1, name='bn_fc1'))
modelo.add(Activation('relu'))
modelo.add(Dense(num_clases, name='salida'))
modelo.add(BatchNormalization(axis=1, name='bn_salida'))

modelo.add(Activation('softmax'))

modelo.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 32, 32, 32)        896       
_________________________________________________________________
bn_conv1 (BatchNormalization (None, 32, 32, 32)        128       
_________________________________________________________________
activation_18 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 30, 30, 32)        9248      
_________________________________________________________________
bn_conv2 (BatchNormalization (None, 30, 30, 32)        128       
_________________________________________________________________
activation_19 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 15, 15, 32)        0         
__________

#### Compilar, Callbacks y Entrenar el modelo

In [ ]:
tamaño_lote = 32
epocas = 200

# inicializar optimizador RMSprop
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Entrenar el modelo usando RMSprop
modelo.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [28]:
# Archivo donde se almacenan los pesos
archivo = 'drive/ColabNotebooks/pesos-v1.{epoch:02d}-{val_loss:.4f}.hdf5'
modelo_chk = ModelCheckpoint(archivo, monitor='val_loss', verbose=0,
                save_best_only=True, save_weights_only=True, mode='auto', period=1)

# Reducir tasa de aprendizaje cuando se estanca el aprendizaje
reducir_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
              patience=5, min_lr=0.001)

# Almacenar los resultados de cada epoca a un archivo CSV
csv_log = CSVLogger('drive/ColabNotebooks/entrenamiento-v1.log')

# Entrenar la Red
modelo.fit_generator(datagen.flow(x_entrenamiento, y_entrenamiento, batch_size=tamaño_lote),
                    steps_per_epoch=x_entrenamiento.shape[0] // tamaño_lote,
                    epochs=epocas,
                    validation_data=(x_prueba, y_prueba),
                    callbacks=[reducir_lr, modelo_chk, csv_log])

Epoch 1/200
1084/1562 [===================>..........] - ETA: 33s - loss: 1.5928 - acc: 0.45001562/1562 [==============================] - 112s 72ms/step - loss: 1.5347 - acc: 0.4756 - val_loss: 1.2971 - val_acc: 0.5675
Epoch 2/200
 276/1562 [====>.........................] - ETA: 1:26 - loss: 1.3603 - acc: 0.54401516/1562 [============================>.] - ETA: 3s - loss: 1.3077 - acc: 0.56801562/1562 [==============================] - 108s 69ms/step - loss: 1.3060 - acc: 0.5687 - val_loss: 1.1941 - val_acc: 0.6050
Epoch 3/200
 427/1562 [=======>......................] - ETA: 1:16 - loss: 1.2380 - acc: 0.59381562/1562 [==============================] - 109s 70ms/step - loss: 1.1995 - acc: 0.6088 - val_loss: 1.1012 - val_acc: 0.6397
Epoch 4/200
  35/1562 [..............................] - ETA: 1:46 - loss: 1.1361 - acc: 0.63041330/1562 [========================>.....] - ETA: 15s - loss: 1.1307 - acc: 0.63481562/1562 [==============================] - 107s 69ms/step - loss: 1.1254 - acc

In [17]:
# cargar los pesos que generaron la mejor exactitud sobre el conjunto de validación 
modelo.load_weights('pesos-v1.188-0.4200.hdf5')

scores = modelo.evaluate(x_prueba, y_prueba, verbose=1)
print('Perdida prueba:', scores[0])
print('Exactitud prueba:', scores[1])

10000/10000 [==============================] - 3s 286us/step
Perdida prueba: 14.511121047973633
Exactitud prueba: 0.0997
